## 1. Catchment

In [1]:
def DrivingTime(longitude,latitude, identify, dt, method = 'distance', vehicle = 'car'):
    """Fungsi untuk melakukan catchment menggunakan driving time sejauh dt
    Parameters
    ----------

    longitude : int
        koordinat x pada point
    latitude : int
        koordinat y pada point
    identify : str
        nama colom sebagai identify data titik (primery key)
    dt : int
        Distance for catchmnet area in meters
    method : str
        menggunakan time atau distance (jika None maka menggunakan default atau distance)
    vehicle : str
        Jenis kendaraan yang digunakan car,truck,atau motorcycle. jika None maka menggunakan default atau car
            
    ----------
    """
    import geopandas as gpd
    import pandas as pd
    
    list_x = [longitude]
    list_y = [latitude]
    list_id = [identify]
        
    list_drivetime = []
    for x, y, z in zip(list_x, list_y, list_id):
        try:
            drivetime = gpd.read_file(os.getenv('API_ISOCHRONE')+"?point={0},{1}&reverse_flow=true&{2}_limit={3}&profile={4}".format(x,y,method,dt,vehicle))
        except:
            drivetime = gpd.read_file(os.getenv('API_ISOCHRONE')+"?point={0},{1}&reverse_flow=true&{2}_limit={3}&profile={4}".format(x,y,method,dt,vehicle))
        drivetime['id'] = z
        list_drivetime.append(drivetime)
    drive_time = pd.concat(list_drivetime).drop(columns='bucket')
    return drive_time

In [2]:
def Buffer(longitude, latitude, dt, identify, Proj='epsg:6933'):
    """Fungsi untuk melakukan catchment menggunakan Buffer sejauh dt
    Parameters
    ----------

    longitude : int
        koordinat x pada point
    latitude : int
        koordinat y pada point
    dt : int
        Distance for catchment area in meters
    identify : str
        nama colom sebagai identify data titik (primery key)
    Proj : str
        Proyeksi dalam geosentrik
    ----------
    """
    import geopandas as gpd
    import pandas as pd
    
    #create dataframe
    df = pd.DataFrame({'id':[identify],
                       'lon':[longitude],
                       'lat':[latitude]})
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs = 'epsg:4326')
    
    #buffering
    buffer         = gpd.GeoDataFrame(gdf.to_crs(Proj).buffer(dt))
    buffer.columns = ['geometry']
    buffer.crs     = Proj
    buffer['id']   = gdf['id']
    buffer         = buffer.to_crs('epsg:4326')
    
    return buffer

## 2. Data Preparation

In [3]:
def Admin_Catch(catch_poly, conn, id_source='6'):
    """Fungsi untuk mendapatkan admin dalam level desa yang tercatchment Buffer atau driving time
    Parameters
    ----------

    catch_poly : geopandas.GeoDataframe
        Polygon cactchment
    conn : engine connection
        engine connection to database
    id_source : int
        id_source yang digunakan untuk data dari database
    ----------
    """
    import pandas as pd
    import geopandas as gpd
    from sqlalchemy import text
    
    sql = text("""select B.id_desa, B.nama_desa, A.id_kota,B.kode_desa, A.kode_kota,
    A.nama_kota, B.geom from tbl_master_desa_gabungan B
    join tbl_master_kota_gabungan A on left(B.kode_desa,4) = A.kode_kota
    where ST_intersects('{0}'::geometry, B.geom) and 
    A.status_data = 'T' and
    B.status_data = 'T' and 
    B.id_source = {1} and 
    A.id_source = {1} """.format(str(catch_poly.dissolve().geometry[0]),id_source))

    admin = gpd.read_postgis(sql, conn, crs='epsg:4326')
    
    return admin

In [4]:
def Demog_Catch(catch_poly, conn, id_source='6', list_variable=['JUMLAH_PEN','PRIA','WANITA',
                                                               'TIDAK_BELU','BELUM_TAMA',
                                                               'TAMAT_SD','SLTP','SLTA',
                                                               'DIPLOMA_I','DIPLOMA_II',
                                                               'STRATA_I','STRATA_II','STRATA_III',
                                                               'U5','010','U15','U20',
                                                               'U25','U30','U35','U40','U45',
                                                               'U50','U55','U60','065','U70']):
    
    """Fungsi untuk mendapatkan admin dalam level desa yang tercatchment Buffer atau driving time
    Parameters
    ----------

    catch_poly : geopandas.GeoDataframe
        Polygon cactchment
    conn : engine connection
        engine connection to database
    id_source : int
        id_source yang digunakan untuk data dari database
    list_variable : list
        list variable demografi yang akan digunakan sebagai analisis
    ----------
    """
    import pandas as pd
    import geopandas as gpd
    import numpy as np
    from sqlalchemy import text
    
    #query for get admin data
    sql = text("""select B.id_desa, B.nama_desa, A.id_kota,B.kode_desa, A.kode_kota,
    A.nama_kota, B.geom from tbl_master_desa_gabungan B
    join tbl_master_kota_gabungan A on left(B.kode_desa,4) = A.kode_kota
    where A.status_data = 'T' and
    B.status_data = 'T' and 
    B.id_source = """+id_source+""" and 
    A.id_source = """+id_source+""" and
    ST_intersects('"""+str(catch_poly.dissolve().geometry[0])+"""'::geometry, B.geom)
     """)

    admin = gpd.read_postgis(sql, conn, crs='epsg:4326')
    
    #select kode_desa
    kode_desa = admin.kode_desa.tolist()
    
    if len(kode_desa)>1:
        #query to get data demografi
        sql = text("""
        select a.id, a.id_source, a.id_desa, a.variabel, b.deskripsi, 
        a.jumlah, a.id_kategori_thematic, a.kode_desa
        from tbl_jml_thematic_gabungan a 
        join tbl_kategori_thematic b on a.variabel = b.variabel
        join tbl_master_desa_gabungan c on c.kode_desa = a.kode_desa
        join tbl_master_kota_gabungan D on left(a.kode_desa,4) = D.kode_kota
        where b.variabel in """+str(tuple(list_variable))+""" and
        a.kode_desa in """+str(tuple(kode_desa))+""" and 
        a.id_source = """+id_source+""" 
        """)
        demog = pd.read_sql(sql, conn)
    else :
        #query to get data demografi
        sql = text("""
        select a.id, a.id_source, a.id_desa, a.variabel, b.deskripsi, 
        a.jumlah, a.id_kategori_thematic, a.kode_desa
        from tbl_jml_thematic_gabungan a 
        join tbl_kategori_thematic b on a.variabel = b.variabel
        join tbl_master_desa_gabungan c on c.kode_desa = a.kode_desa
        join tbl_master_kota_gabungan D on left(a.kode_desa,4) = D.kode_kota
        where b.variabel in """+str(tuple(list_variable))+""" and
        a.kode_desa ="""+str(kode_desa[0])+""" and 
        a.id_source = """+id_source+""" 
        """)
        demog = pd.read_sql(sql, conn)
    #drop duplicates dari hasil query
    demog = demog.drop_duplicates()
    
    #Unmelted demographic table
    demog_unmelted = demog.pivot(index='kode_desa', columns='variabel', values='jumlah')
    demog_unmelted = demog_unmelted.reset_index()
    demog_unmelted.columns.name = None
    
    #join with gdf admin
    demog_join = pd.merge(admin,demog_unmelted,on='kode_desa',how='left')
    gdf_demog  = demog_join.replace(np.NaN, 0)
    
    return demog_join

In [5]:
def SES_Catch(catch_poly, conn, id_source='6'):
    """Fungsi untuk mendapatkan admin dalam level desa yang tercatchment Buffer atau driving time
    Parameters
    ----------

    catch_poly : geopandas.GeoDataframe
        Polygon cactchment
    conn : engine connection
        engine connection to database
    id_source : int
        id_source yang digunakan untuk data dari database
    ----------
    """
    import pandas as pd
    import geopandas as gpd
    from sqlalchemy import text
    
    sql = text("""
    select id_desa, nilai, b.geom from tbl_ses_gabungan a
    left join tbl_master_desa_gabungan b using(id_desa)
    where a.id_source = """+id_source+""" and b.id_source = """+id_source+""" and 
    ST_intersects('"""+str(catch_poly.dissolve().geometry[0])+"""'::geometry, b.geom) 
    """)
    gdf_ses = gpd.read_postgis(sql,conn0, crs='epsg:4326')
    
    return gdf_ses

In [6]:
#Ubah POI berdasarkan intersect
def POI_Catch(catch_poly, conn, col_name = None, category = []):
    """Fungsi untuk mendapatkan admin dalam level desa yang tercatchment Buffer atau driving time
    Parameters
    ----------

    conn : engine connection
        engine connection to database
    catch_poly : geopandas.GeoDataframe
        Polygon cactchment
    category : list
        list nama kategori poi
    ----------
    """
    import pandas as pd
    import geopandas as gpd
    from sqlalchemy import text
    
    #query untuk get data POI
    sql = text("""
    select id_merchant, nama_merchant, a.kode_brand, a.nama_brand, a.nama_sub_kategori,
    a.kode_sub_kategori,nama_kategori,kode_kategori, geom 
        from (select id_merchant, nama_merchant, a.kode_brand, a.nama_brand, a.nama_sub_kategori,a.kode_sub_kategori, geom, kode_kategori, nama_kategori
              from (select id_merchant, nama_merchant, B.kode_brand, B.nama_brand, C.kode_sub_kategori, C.nama_sub_kategori, geom, D.kode_kategori, D.nama_kategori
                    from (select id_merchant, id_desa,id_brand, nama_merchant, kode_desa, geom 
                          from tbl_merchant where status = 'T' 
                          and ST_intersects('"""+str(catch_poly.dissolve().geometry[0])+"""'::geometry, geom)) Z
                          left join tbl_brand B on B.id_brand = Z.id_brand
                          left join tbl_sub_kategori C on B.id_sub_kategori = C.id_sub_kategori 
                          left join tbl_kategori D on C.id_kategori = D.id_kategori and 
                          B.status_data = 'T' and C.status_data = 'T' and D.status_data = 'T')a)a

    """)
    poi_all = gpd.read_postgis(sql, conn0, crs = 'epsg:4326')
    
    if len(category)>0:
        poi_category = poi_all[poi_all[col_name].isin(category)]
        gdf_poi = gpd.sjoin(poi_category,catch_poly[['geometry']]).drop(columns = 'index_right')
    else:
        gdf_poi = gpd.sjoin(poi_all,catch_poly[['geometry']]).drop(columns = 'index_right')
        
        
    return gdf_poi

In [55]:
def StreetCatch(catch_poly, conn):
    """Fungsi untuk mendapatkan data jalan tercatchment Buffer atau driving time
    Parameters
    ----------
    conn : engine connection
        engine connection to database
    catch_poly : geopandas.GeoDataframe
        Polygon cactchment
    ----------
    """
    
    import pandas as pd
    import geopandas as gpd
    import numpy as np
    from sqlalchemy import text
    
    sql = text("""
    SELECT objectid, name_id, fromnode, tonode, 
    road_cls, width, nopass_c, oneway, lanes, 
    max_speed, sepa_cls, road_fas, roadcon, road_attr, 
    road_name, st_setsrid(geom,4326) as geom  
    FROM road.roadlink_up 
    Where ST_intersects(st_setsrid('"""+str(catch_poly.dissolve().geometry[0])+"""'::geometry,4326), geom)
    """)
    jalan = gpd.read_postgis(sql, conn)
    
    jalan['road classification'] = np.where(jalan['road_cls']=='0101','National Primary Road',
        np.where(jalan['road_cls']=='0102','National Secondary Road',
                np.where(jalan['road_cls']=='0103','Arterial-Collector Road',
                        np.where(jalan['road_cls']=='0204','Toll Road',
                                np.where(jalan['road_cls']=='0305','Local Primary Road',
                                        np.where(jalan['road_cls']=='0306','Local Secondary Road','Not Classify'))))))
    return jalan

## 3. Data Visualization

In [7]:
def LokasiCatchmentMap(catch_poly, conn, longitude, latitude, map_name='Map_Lokasi_Catchment', id_source='6'):
    """Fungsi untuk mendapatkan visualisasi LOKASI catchment yang dioverlay terhadap administrasi dalam level desa
    Parameters
    ----------

    catch_poly : geopandas.GeoDataframe
        Polygon cactchment
    conn : engine connection
        engine connection to database
    longitude : int
        koordinat x pada point
    latitude : int
        koordinat y pada point
    map_name : str
        nama map yang akan disimpan kedalam format png
    id_source : int
        id_source yang digunakan untuk data dari database
    ----------
    """
    import geopandas as gpd
    import pandas as pd
    import contextily as cx
    import matplotlib.pyplot as plt
    from sqlalchemy import text
    
    #query for get admin data
    sql = text("""select B.id_desa, B.nama_desa, A.id_kota,B.kode_desa, A.kode_kota,
    A.nama_kota, B.geom from tbl_master_desa_gabungan B
    join tbl_master_kota_gabungan A on left(B.kode_desa,4) = A.kode_kota
    where ST_intersects('"""+str(catch_poly.dissolve().geometry[0])+"""'::geometry, B.geom) and 
    A.status_data = 'T' and
    B.status_data = 'T' and 
    B.id_source = """+id_source+""" and 
    A.id_source = """+id_source+""" """)

    admin = gpd.read_postgis(sql, conn, crs='epsg:4326')
    
    
    #create dataframe
    df = pd.DataFrame({'lon':[longitude],
                       'lat':[latitude]})
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs = 'epsg:4326')
    
    
    print('Catchment terletak di '+str(admin.nama_kota.nunique())+' kota yaitu '+str(tuple(admin.nama_kota.unique()))+
          ' dan terdiri dari '+str(admin.id_desa.nunique())+' Desa')
    
    fig,ax = plt.subplots(figsize=(10, 15)) 
    admin.to_crs(epsg=3857).plot(ax=ax,alpha=0.5, 
               edgecolor='grey', 
               color='cornsilk')
    admin.boundary.to_crs(epsg=3857).plot(ax=ax,alpha=0.7, edgecolor='grey',linestyle = 'dashed', label = 'Administration')
    catch_poly.boundary.to_crs(epsg=3857).plot(ax=ax,
                                color='red', label = 'Catchment Boundary')
    gdf.to_crs(epsg=3857).plot(ax=ax,color = 'black', markersize = 100, marker='X', label = 'Location of Analysis')
    ax.set_xlim((catch_poly.to_crs(epsg=3857).bounds['minx'][0]-4000),(catch_poly.to_crs(epsg=3857).bounds['maxx'][0]+1000))
    ax.set_ylim((catch_poly.to_crs(epsg=3857).bounds['miny'][0]-3000),(catch_poly.to_crs(epsg=3857).bounds['maxy'][0]+3000))
    cx.add_basemap(ax=ax, 
                   source=cx.providers.CartoDB.Positron)
    #ax.set_axis_off()
    plt.savefig(''+map_name+'.png')
    ax.legend(fontsize = 12,
             frameon = True,
             loc = 4,
             title = 'LEGEND')
     
    #plt.close()

In [8]:
def POICatchmentMap(gdf_POI, catch_poly, longitude, latitude, col_class='nama_kategori', col_name = None, category = [], map_name='Map_POI_Catchment'):
    """Fungsi untuk mendapatkan visualisasi POI dalam catchment
    Parameters
    ----------
    gdf_POI  : geopandas.GeoDataFrame
        POI berupa Point dalam catchment
    catch_poly : geopandas.GeoDataframe
        Polygon catchment
    longitude : int
        koordinat x pada point
    latitude : int
        koordinat y pada point
    map_name : str
        nama map yang akan disimpan kedalam format png
    col_class : str
        nama kolom yang digunakan sebagai kategori (Class)\
    col_name : str
        nama kolom yang digunakan sebagai seleksi data (nama/kode kategori, nama/kode sub_kategori, nama/kode brand)
    category : list of str
        list nama kategori/sub_kategori/brand yang akan dilakukan seleksi
    ----------
    """
    import geopandas as gpd
    import pandas as pd
    import contextily as cx
    import matplotlib.pyplot as plt
    
    #create dataframe
    df = pd.DataFrame({'lon':[longitude],
                       'lat':[latitude]})
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs = 'epsg:4326')
    
    if len(category)>0:
        poi = gdf_POI[gdf_POI[col_name].isin(category)]
    else :
        poi = gdf_POI
        
    #generate map
    fig,ax = plt.subplots(figsize=(10, 15)) 
    catch_poly.to_crs(epsg=3857).plot(ax=ax,
                                 color='cornsilk')
    catch_poly.boundary.to_crs(epsg=3857).plot(ax=ax, 
                                          color='red',
                                          label = 'Catchment Boundary')
    gdf.to_crs(epsg=3857).plot(ax=ax,
                               color = 'black', 
                               markersize = 100, 
                               marker='X', 
                               label = 'Location of Analysis')
    for ctype, data in poi.groupby(col_class):
        label = ctype
        data.to_crs(epsg=3857).plot(ax=ax,
                                    label = label,
                                    markersize = 50)
    ax.set_xlim((catch_poly.to_crs(epsg=3857).bounds['minx'][0]-800),(catch_poly.to_crs(epsg=3857).bounds['maxx'][0]+2800))
    ax.set_ylim((catch_poly.to_crs(epsg=3857).bounds['miny'][0]-800),(catch_poly.to_crs(epsg=3857).bounds['maxy'][0]+800))
    cx.add_basemap(ax=ax, 
                   source=cx.providers.CartoDB.Positron)
    ax.legend(fontsize = 12,
             frameon = True,
             loc = 4,
             title = 'LEGEND')
    
    #ax.set_axis_off()
    plt.savefig(''+map_name+'.png')
    #plt.close()

In [23]:
def POIDistanceMap(gdf_POI, catch_poly, longitude, latitude, col_class='nama_kategori', col_name = None, category = [], map_name='Map_POI_Catchment'):
    """Fungsi untuk mendapatkan visualisasi distance/heatmap POI dalam catchment
    Parameters
    ----------
    gdf_POI  : geopandas.GeoDataFrame
        POI berupa Point dalam catchment
    catch_poly : geopandas.GeoDataframe
        Polygon catchment
    longitude : int
        koordinat x pada point
    latitude : int
        koordinat y pada point
    map_name : str
        nama map yang akan disimpan kedalam format png
    col_class : str
        nama kolom yang digunakan sebagai kategori (Class)\
    col_name : str
        nama kolom yang digunakan sebagai seleksi data (nama/kode kategori, nama/kode sub_kategori, nama/kode brand)
    category : list of str
        list nama kategori/sub_kategori/brand yang akan dilakukan seleksi
    ----------
    """
    import geopandas as gpd
    import pandas as pd
    import contextily as cx
    import matplotlib.pyplot as plt
    from shapely.geometry import LineString
    import plotly.express as px
    import plotly.graph_objs as go
    
    #create dataframe
    df = pd.DataFrame({'lon':[longitude],
                       'lat':[latitude]})
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs = 'epsg:4326')
    
    if len(category)>0:
        poi = gdf_POI[gdf_POI[col_name].isin(category)]
    else :
        poi = gdf_POI
    
    if poi.shape[0] < 500:
        gdf[['latitude', 'longitude']] = gdf.apply(
            lambda p: (p.geometry.y, p.geometry.x), axis=1, result_type='expand')
        poi[['latitude', 'longitude']] = poi.apply(
            lambda p: (p.geom.y, p.geom.x), axis=1, result_type='expand')
        list_dist = []
        list_polyline = []
        for i in poi.id_merchant:
            x = poi[poi['id_merchant']==i].reset_index()
            distance = gdf.to_crs(Proj).centroid.distance(x.to_crs(Proj).centroid[0])[0]
            lines = LineString([(gdf.longitude.values[0], gdf.latitude.values[0]),
                                            (x.longitude[0], x.latitude[0])])
            list_dist.append(distance)
            list_polyline.append(lines)
        gdf_line = poi[['id_merchant']].copy()
        gdf_line['polyline'] = list_polyline
        gdf_line['distance'] = list_dist
        gdf_distance = gpd.GeoDataFrame(gdf_line, geometry=gdf_line['polyline'])
        del gdf_distance['polyline']
        
        #generate map
        fig,ax = plt.subplots(figsize=(10, 15)) 
        catch_poly.to_crs(epsg=3857).plot(ax=ax,
                                     color='cornsilk')
        catch_poly.boundary.to_crs(epsg=3857).plot(ax=ax, 
                                              color='red',
                                              label = 'Catchment Boundary')
        gdf_distance.to_crs(epsg=3857).plot(ax=ax,
                                            color = 'white',
                                            label = 'Euclidean distance')
        gdf.to_crs(epsg=3857).plot(ax=ax,
                                   color = 'black', 
                                   markersize = 100, 
                                   marker='X', 
                                   label = 'Location of Analysis')
        for ctype, data in poi.groupby(col_class):
            label = ctype
            data.to_crs(epsg=3857).plot(ax=ax,
                                        label = label,
                                        markersize = 50)
        ax.set_xlim((catch_poly.to_crs(epsg=3857).bounds['minx'][0]-800),(catch_poly.to_crs(epsg=3857).bounds['maxx'][0]+2800))
        ax.set_ylim((catch_poly.to_crs(epsg=3857).bounds['miny'][0]-800),(catch_poly.to_crs(epsg=3857).bounds['maxy'][0]+800))
        cx.add_basemap(ax=ax, 
                       source=cx.providers.CartoDB.Positron)
        ax.legend(fontsize = 12,
                 frameon = True,
                 loc = 4,
                 title = 'LEGEND')

        #ax.set_axis_off()
        plt.savefig(''+map_name+'_distance.png')
        #plt.close()
    else :
        gdf_POI['count'] = 1
        gdf_POI['longitude'] = gdf_POI.geometry.x
        gdf_POI['latitude'] = gdf_POI.geometry.y
        fig = px.density_mapbox(gdf_POI, lat='latitude', lon='longitude', z='count',
                        mapbox_style='carto-positron', zoom=11)
        fig.add_trace(
            go.Scattermapbox(
                lat=gdf["lat"],
                lon=gdf["lon"],marker =dict(color='red',size = 20),
                mode="markers",
                showlegend=False,
                hoverinfo="skip",

            )
        )
        fig
        
    return fig
        

In [26]:
def POICatchmentChart(gdf_POI, col_class='nama_kategori', chart_name='Chart_POI_Catchment', col_name = None, category = []):
    """Fungsi untuk mendapatkan Chart POI dalam catchment
    Parameters
    ----------
    gdf_POI  : geopandas.GeoDataFrame
        POI berupa Point dalam catchment
    chart_name : str
        nama chart yang akan disimpan kedalam format png
    col_class : str
        nama kolom yang digunakan sebagai kategori (Class)\
    col_name : str
        nama kolom yang digunakan sebagai seleksi data (nama/kode kategori, nama/kode sub_kategori, nama/kode brand)
    category : list of str
        list nama kategori/sub_kategori/brand yang akan dilakukan seleksi
    ----------
    """
    import geopandas as gpd
    import pandas
    import plotly.express as px
    from selenium import webdriver
    import time
    import os
    
    if len(category)>0:
        gdf_POI = gdf_POI[gdf_POI[col_name].isin(category)]
    else :
        gdf_POI = gdf_POI
    
    group = gdf_POI.groupby(col_class)['id_merchant'].count().reset_index()
    group.rename(columns = {'id_merchant':'Total_POI'}, inplace = True)
    fig = px.bar(group.sort_values('Total_POI', ascending=False), 
                     x='Total_POI', 
                     y=col_class, 
                     text_auto=True,
                     title="Total POI in Catchment Classification by "+str(col_class),
                    orientation = 'h')
    fig.update_traces(marker_color='#3A8891', 
                          marker_line_color='#3C6255',
                          marker_line_width=1.5, opacity=0.6)
    fig.update_layout(yaxis_title=col_class, 
                          xaxis_title="Total POI")
    fig.show()
    fig.write_html(''+chart_name+'.html')
    delay =5 
    tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=''+chart_name+'.html')
    browser = webdriver.Chrome()
    browser.get(tmpurl)

In [10]:
def SESCatchmentMap(gdf_SES, catch_poly, longitude, latitude, map_name='Map_SES_Catchment'):
    """Fungsi untuk mendapatkan visualisasi SES dalam catchment
    Parameters
    ----------
    gdf_SES  : geopandas.GeoDataFrame
        Administration geom dalam catchment dengan nilai SES
    catch_poly : geopandas.GeoDataframe
        Polygon catchment
    longitude : int
        koordinat x pada point
    latitude : int
        koordinat y pada point
    map_name : str
        nama map yang akan disimpan kedalam format png
    ----------
    """
    import geopandas as gpd
    import pandas as pd
    import contextily as cx
    import matplotlib.pyplot as plt
    
    #create dataframe
    df = pd.DataFrame({'lon':[longitude],
                       'lat':[latitude]})
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs = 'epsg:4326')
    
    #clip admin SES with catchment area
    ses_clip = gpd.clip(gdf_SES, catch_poly[['geometry']])
    
    #generate map
    fig,ax = plt.subplots(figsize=(10, 15)) 
    pointsPalette = {'High':'#3a8891', 
              'Medium-High':'#7dacaa', 
              'Medium-Low':'#b9cdc1', 
              'Low':'#f5edd7'}
    for ctype, data in ses_clip.groupby('nilai'):
        color = pointsPalette[ctype]
        label = ctype
        data.to_crs(epsg=3857).plot(ax=ax,
                                    color = color,
                                    label = label)
    for ctype, data in ses_clip.groupby('nilai'):
        color = pointsPalette[ctype]
        label = ctype
        data.centroid.to_crs(epsg=3857).plot(ax=ax,
                                    color = color,
                                    label = label,
                                    markersize = 50)
    catch_poly.boundary.to_crs(epsg=3857).plot(ax=ax, 
                                          color='red',
                                          label = 'Catchment Boundary')
    gdf.to_crs(epsg=3857).plot(ax=ax,
                               color = 'black', 
                               markersize = 100, 
                               marker='X', 
                               label = 'Location of Analysis')
    
    ax.set_xlim((catch_poly.to_crs(epsg=3857).bounds['minx'][0]-800),(catch_poly.to_crs(epsg=3857).bounds['maxx'][0]+800))
    ax.set_ylim((catch_poly.to_crs(epsg=3857).bounds['miny'][0]-3000),(catch_poly.to_crs(epsg=3857).bounds['maxy'][0]+800))
    cx.add_basemap(ax=ax, 
                   source=cx.providers.CartoDB.Positron)
    ax.legend(fontsize = 12,
             frameon = True,
             loc = 4,
             title = 'LEGEND')
    
    #ax.set_axis_off()
    plt.savefig(''+map_name+'.png')
    #plt.close()

In [11]:
def SESCatchmentChart(gdf_SES, catch_poly, chart_name = 'Chart_SES_Catchment', Proj = 'epsg:6933' ):
    """Fungsi untuk mendapatkan Chart SES dalam catchment
    Parameters
    ----------
    gdf_SES  : geopandas.GeoDataFrame
        Administration geom dalam catchment dengan nilai SES
    catch_poly : geopandas.GeoDataframe
        Polygon catchment
    chart_name : str
        nama chart yang akan disimpan kedalam format png
    Proj : str
        Proyeksi dalam geosentrik
    ----------
    """
    import webdriver
    import time
    import geopandas as gpd
    import pandas as pd
    import plotly.express as px
    from selenium
    import os
    
    #Calculate percentage of SES
    percentage = pd.DataFrame()
    percentage['id_desa'] = []
    percentage['percent'] = []
    x = gpd.clip(gdf_SES, catch_poly).reset_index()
    percentage['id_desa'] = x.id_desa.unique()
    percentage['percent'] = x.to_crs(Proj).area/catch_poly.to_crs(Proj).area[0]

    ses_merge = pd.merge(percentage,ses[['id_desa','nilai']])
    
    #SES Chart
    group = ses_merge.groupby('nilai')['percent'].sum().reset_index()
    group['percent'] = round(group['percent']*100,2)
    group.columns = ['nilai','percentage']
    if group.nilai.nunique()>2:
        fig = px.bar(group, 
                     y='percentage', 
                     x='nilai', 
                     text_auto=True,
                     title="SES Percentage in Catchment")
        fig.update_xaxes(categoryorder='array', 
                             categoryarray= ['High','Medium-High','Medium-Low','Low'])
        fig.update_traces(marker_color='#3A8891', 
                          marker_line_color='#3C6255',
                          marker_line_width=1.5, opacity=0.6)
        fig.update_layout(xaxis_title="Sub Category", 
                          yaxis_title="Percentage",
                         yaxis_range=[0,100])
        fig.show()
        fig.write_html(''+chart_name+'.html')
        delay =5 
        tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=''+chart_name+'.html')
        browser = webdriver.Chrome()
        browser.get(tmpurl)
        #Give the map tiles some time to load
        time.sleep(delay)
        browser.save_screenshot(''+chart_name+'.png')
        browser.quit()
    else:
        night_colors = ['#f5edd7', '#3A8891']
        labels = group.nilai.tolist()
        values = group.percentage.tolist()
        fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3,marker_colors=night_colors,textinfo='percent+label'
                                     )])
        fig.update_layout(title = "SES Percentage in Catchment")
        fig.show()
        fig.write_html(''+chart_name+'.html')
        delay =5 
        tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=''+chart_name+'.html')
        browser = webdriver.Chrome()
        browser.get(tmpurl)
        #Give the map tiles some time to load
        time.sleep(delay)
        browser.save_screenshot(''+chart_name+'.png')
        browser.quit()

In [12]:
def DemographyCatchmentChart(gdf_demog, catch_poly, variable, Proj = 'epsg:6933', chart_name = 'Chart_Demog_Catchment'):
    """Fungsi untuk mendapatkan Chart Demography dalam catchment
    Parameters
    ----------
    gdf_demog  : geopandas.GeoDataFrame
        Administration geom dalam catchment dengan nilai SES
    catch_poly : geopandas.GeoDataframe
        Polygon catchment
    variable : list
        list variable demografi yang akan digunakan dalam chart
    chart_name : str
        nama chart yang akan disimpan kedalam format png
    Proj : str
        Proyeksi dalam geosentrik
    ----------
    """
    import webdriver
    import time
    import geopandas as gpd
    import pandas as pd
    import plotly.express as px
    from selenium
    import os
    
    #calculate percentage of demog in catchment
    percentage = pd.DataFrame()
    percentage['kode_desa'] = []
    percentage['percent'] = []
    x = gpd.clip(gdf_demog, catch_poly).reset_index()
    y = gdf_demog.copy()
    percentage['kode_desa'] = x.kode_desa.unique()
    percentage['percent'] = x.to_crs(Proj).area/y.to_crs(Proj).area
    
    percentage['kode_desa'] = percentage['kode_desa'].astype(str)
    demog_percent = pd.merge(percentage, gdf_demog.drop(columns = ['id_desa', 'nama_desa', 
                                                           'id_kota', 'nama_kota', 'geom']), on = 'kode_desa')
    demog_cal = pd.DataFrame()
    
    columns = demog_percent.columns.tolist()
    variable = variable
    var = []
    not_var = []
    for i in variable:
        if i in columns:
            var.append(i)
        else:
            not_var.append(i)
            
    for i in var:
        demog_cal[i] = round(demog_percent['percent']*demog_percent[i])
        
    calculate = demog_cal.sum().reset_index()
    calculate.columns = ['variable','Population']
    
    if calculate.variable.nunique()>2:
        fig = px.bar(calculate, 
                     y='Population', 
                     x='variable', 
                     text_auto=True,
                     title="Total Population in Catchment")
        fig.update_xaxes(categoryorder='array', 
                             categoryarray= ['TAMAT_SD','SLTP','SLTA', 'DIPLOMA_I', 'DIPLOMA_II', 
                                             'STRATA_I', 'STRATA_II', 'STRATA_III', 'U5','U10',
                                             'U15','U20', 'U25', 'U30', 'U40', 'U45', 'U50', 
                                             'U55','U60','U65','U70','U75'])
        fig.update_traces(marker_color='#3A8891', 
                          marker_line_color='#3C6255',
                          marker_line_width=1.5, opacity=0.6)
        fig.update_layout(xaxis_title="Variable", 
                          yaxis_title="Total Population")
        fig.show()
        fig.write_html(''+chart_name+'.html')
        delay =5 
        tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=''+chart_name+'.html')
        browser = webdriver.Chrome()
        browser.get(tmpurl)
        #Give the map tiles some time to load
        time.sleep(delay)
        browser.save_screenshot(''+chart_name+'.png')
        browser.quit()
    else:
        night_colors = ['#f5edd7', '#3A8891']
        labels = calculate.variable.tolist()
        values = calculate.Population.tolist()
        fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3,marker_colors=night_colors,textinfo='value+label+percent'
                                     )])
        fig.update_layout(title = "Total Population in Catchment")
        fig.show()
        fig.write_html(''+chart_name+'.html')
        delay =5 
        tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=''+chart_name+'.html')
        browser = webdriver.Chrome()
        browser.get(tmpurl)
        #Give the map tiles some time to load
        time.sleep(delay)
        browser.save_screenshot(''+chart_name+'.png')
        browser.quit()
        

In [116]:
def StreetCatchmentMap(gdf_street, catch_poly, longitude, latitude, map_name='Map_Street_Catchment'):
    """Fungsi untuk mendapatkan visualisasi Jalan dalam catchment
    Parameters
    ----------
    gdf_street  : geopandas.GeoDataFrame
        Street geom dalam catchment dengan nilai SES
    catch_poly : geopandas.GeoDataframe
        Polygon catchment
    longitude : int
        koordinat x pada point
    latitude : int
        koordinat y pada point
    map_name : str
        nama map yang akan disimpan kedalam format png
    ----------
    """
    import geopandas as gpd
    import pandas as pd
    import contextily as cx
    import matplotlib.pyplot as plt
    
    #create dataframe
    df = pd.DataFrame({'lon':[longitude],
                       'lat':[latitude]})
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs = 'epsg:4326')
    
    
    #generate map
    fig,ax = plt.subplots(figsize=(10, 15))
    pointsPalette = {'National Primary Road':'#45065A', 
                      'National Secondary Road':'#3B508A', 
                      'Arterial-Collector Road':'#287A8E', 
                      'Toll Road':'#2AB07E',
                    'Local Primary Road':'#95D73F',
                     'Local Secondary Road':'#F8E621'
                    }
    catch_poly.boundary.to_crs(epsg=3857).plot(ax=ax, 
                                          color='grey',
                                          label = 'Catchment Boundary')
    for ctype, data in gdf_street.groupby('road classification'):
        color = pointsPalette[ctype]
        label = ctype
        data.to_crs(epsg=3857).plot(ax=ax,
                                          color=color,
                                    label = label)
    
    gdf.to_crs(epsg=3857).plot(ax=ax,
                               color = 'black', 
                               markersize = 100, 
                               marker='X', 
                               label = 'Location of Analysis')
    
    ax.set_xlim((catch_poly.to_crs(epsg=3857).bounds['minx'][0]-800),(catch_poly.to_crs(epsg=3857).bounds['maxx'][0]+6500))
    ax.set_ylim((catch_poly.to_crs(epsg=3857).bounds['miny'][0]-800),(catch_poly.to_crs(epsg=3857).bounds['maxy'][0]+800))
    cx.add_basemap(ax=ax, 
                   source=cx.providers.CartoDB.Positron)
    ax.legend(fontsize = 8,
             frameon = True,
             loc = 4,
             title = 'LEGEND')
    plt.savefig(''+map_name+'.png')

In [123]:
def StreetCatchmentChart(gdf_street, chart_name='Chart_Street_Catchment'):
    """Fungsi untuk mendapatkan Chart Jalan dalam catchment berdasarkan kelas jalan
    Parameters
    ----------
    gdf_POI  : geopandas.GeoDataFrame
        POI berupa Point dalam catchment
    chart_name : str
        nama chart yang akan disimpan kedalam format png
    ----------
    """
    import geopandas as gpd
    import pandas
    import plotly.express as px
    from selenium import webdriver
    import time
    import os
    
    group = gdf_street.groupby('road classification')['objectid'].count().reset_index()
    group.rename(columns = {'objectid':'Total_Jalan'}, inplace = True)
    fig = px.bar(group.sort_values('Total_Jalan', ascending=False), 
                     x='Total_Jalan', 
                     y='road classification', 
                     text_auto=True,
                     title="Total Segment in Catchment Classification based on Road Classification ",
                    orientation = 'h')
    fig.update_traces(marker_color='#3A8891', 
                          marker_line_color='#3C6255',
                          marker_line_width=1.5, opacity=0.6)
    fig.update_layout(yaxis_title='Road Classification', 
                          xaxis_title="Total POI")
    fig.show()
    fig.write_html(''+chart_name+'.html')
    delay =5 
    tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=''+chart_name+'.html')
    browser = webdriver.Chrome()
    browser.get(tmpurl)